heartbeat thump?


In [ ]:
var fs = require('fs');

function thump(thump) {
    var out = fs.openSync('/tmp/myjob.log', 'a');
    var err = fs.openSync('/tmp/myjob.log', 'a');

    require('child_process').spawn('/usr/local/bin/node', [
        '-e',
        'require("/Users/briancullinan/jupytangular2/Core").import("' + thump + '")().catch(e => console.log(e)).then(() => process.exit())'
    ], {
        detached: true,
        stdio: [ 'ignore', out, err ]
    }).unref()
    process.exit();
}
module.exports = thump;
